# Germany

In [2]:
import re
import pandas as pd
import geopandas as gpd

pd.options.mode.chained_assignment = None

## Fetch results

In [4]:
with open ('../../data/raw/deu.csv') as f:
    text = f.read()

In [5]:
data = []
for row in text.split('\n'):
    kreis_nr = re.findall('\d+ (\d+ \d+ \d+)', row)[0].strip()
    kreis = re.findall('\d+ \d+ \d+ \d+ +(\D+)', row)[0].strip()
    turnout = re.findall('\d+,\d', row)[0].strip()
    data.append((kreis_nr, kreis, turnout))

In [6]:
df = pd.DataFrame(data)

In [7]:
df.columns = ['id', 'kreis', 'turnout']

In [8]:
df.id = df.id.str.replace(' ', '')
df.kreis = df.kreis.str[:-3]
df.turnout = df.turnout.str.replace(',', '.').astype(float)

## Merge

In [20]:
geo = gpd.read_file('../../data/shapefiles/deu/gadm36_DEU_2.shp')

In [21]:
# Oops, saved old turnouts in shapefile via QGIS
geo = geo.drop('turnout', axis=1)

In [22]:
geo = geo.merge(df, left_on='CC_2', right_on='id', how='left')

In [25]:
geo = geo[['NAME_2', 'turnout', 'geometry']]
geo = geo.rename(columns={'NAME_2': 'name'})
geo['country'] = 'Germany'

## Export

In [27]:
geo.to_file('../../data/processed/deu')